# Surreact

###  Data modifiaction 

In [ ]:
import os
import tqdm
import numpy as np

avatar_count = 20000
counter = 0

for number in tqdm.tqdm(range(avatar_count)):
    # if number == 5:
    #     continue
    file = (tuple(np.load(f"../Surreact-APose/test/bodymeasurements/Avatar_{number:06d}.npy", allow_pickle=True)))
    measurements = []
    for value in file:
        measurements.append(float(value * 100))
    np.save(f"../Surreact-APose/test/measurementsCM/Avatar_{counter:06d}.npy", measurements)
    counter += 1

### Image modification

In [ ]:
import os
import cv2
import numpy as np
import tqdm


avatar_count = 20000
counter = 0

for number in range(avatar_count):
    image = cv2.imread(f"../Surreact-APose/test/imgs_nobg_side/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
    alpha_channel = image[:, :, 3]
    body_indices = alpha_channel > 30
    image[body_indices] = [255, 255, 255, 255]
    indices = np.where(body_indices)
    leftmost_pixel = np.min(indices[1])
    rightmost_pixel = np.max(indices[1])
    topmost_pixel = max(np.min(indices[0]), 0)
    bottommost_pixel = min(np.max(indices[0]), 239)
    cropped = image[topmost_pixel:bottommost_pixel, leftmost_pixel:rightmost_pixel]

    border_height = 320 - cropped.shape[0]
    border_width = 240 - cropped.shape[1]

    # Calculate the border sizes
    top_border = border_height // 2
    bottom_border = border_height - top_border
    left_border = border_width // 2
    right_border = border_width - left_border

    # Add the border
    bordered_image = cv2.copyMakeBorder(cropped, top_border, bottom_border, left_border, right_border, cv2.BORDER_CONSTANT, value=(0,0,0,0))
    bordered_image = cv2.cvtColor(bordered_image, cv2.COLOR_RGBA2GRAY)


    cv2.imwrite(f"./Export/Surreact-APose/test/images_side/Avatar_{counter:06d}.png", bordered_image)
    counter +=1

In [ ]:
# import cv2
# 
# for number in range(20000):
#     image = cv2.imread(f"Export/Surreact-APose/test/images_side/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     image = cv2.flip(image, 1)
#     cv2.imwrite(f"Export/Surreact-APose/test/images_side/Avatar_{number:06d}.png", image)

# BodyM

### All data

In [ ]:
import csv
import numpy as np
import os
from collections import defaultdict
import cv2
import tqdm.notebook


for folder in tqdm.notebook.tqdm(["testA","testB"]):
    counter = 0
    with open('../bodym-dataset/'+folder+'/subject_to_photo_map.csv', 'r') as f:
        reader = csv.reader(f)
        uuid_raw = list(reader)


    uuid_map = defaultdict(list)

    for subject_id, photo_id in uuid_raw[1:]:
        uuid_map[subject_id].append(photo_id)
    image_data = {}
    hwg_data = {}

    with open('../bodym-dataset/'+folder+'/measurements.csv', 'r') as f:
        reader = csv.reader(f)

        for line in reader:
            subject_id = line[0]


            for image_id in uuid_map[subject_id]:
                image_data[image_id] = np.array(list(map(float, line[1:])))

    with open('../bodym-dataset/'+folder+'/hwg_metadata.csv', 'r') as f:
        reader = csv.reader(f)

        for line in reader:
            subject_id = line[0]
            hwg_data[subject_id] = np.array(line[2:])


    for subject_id, images in tqdm.notebook.tqdm(uuid_map.items()):
        for image in images:
            measurements = np.append(image_data[image], float(hwg_data[subject_id][0]))
            measurements = np.array(measurements)
            silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
            silhouette = cv2.resize(silhouette,(240, 320))
            cv2.imwrite(os.path.join(f'Export/BodyM/{folder}/images_front/Avatar_{counter:06d}.png'), silhouette)
            np.save(os.path.join(f'Export/BodyM/{folder}/measurements/Avatar_{counter:06d}.npy'), measurements)


            silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask_left/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
            silhouette = cv2.resize(silhouette,(240, 320))
            cv2.imwrite(os.path.join(f'Export/BodyM/{folder}/images_side/Avatar_{counter:06d}.png'), silhouette)
            counter += 1

In [ ]:
# #FOR BODY_M DATASET
# import csv
# import numpy as np
# import os
# from collections import defaultdict
# import cv2
# import tqdm.notebook
# 
# counter = 0
# 
# os.remove("BodyM_Conversion.csv")
# for folder in tqdm.notebook.tqdm(["testA", "testB", "train"]):
# 
#     with open('../bodym-dataset/'+folder+'/subject_to_photo_map.csv', 'r') as f:
#         reader = csv.reader(f)
#         uuid_raw = list(reader)
#     
# 
#     uuid_map = defaultdict(list)
# 
#     for subject_id, photo_id in uuid_raw[1:]:
#         uuid_map[subject_id].append(photo_id)
#     image_data = {}
#     hwg_data = {}
# 
#     with open('../bodym-dataset/'+folder+'/measurements.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
# 
# 
#             for image_id in uuid_map[subject_id]:
#                 image_data[image_id] = np.array(list(map(float, line[1:])))
# 
#     with open('../bodym-dataset/'+folder+'/hwg_metadata.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
#             hwg_data[subject_id] = np.array(line[2:])
# 
# 
# 
#     
#     with open("BodyM_Conversion.csv", "a", newline="") as file:
#         writer = csv.writer(file, delimiter=";")
#         writer.writerow(["subject_id","image_id","measurements","hwg_data","avatar_id"])
#         for subject_id, images in tqdm.notebook.tqdm(uuid_map.items()):
#             for image in images:
#                 measurements = np.array(image_data[image])
#                 silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#                 silhouette = cv2.resize(silhouette,(240, 320))
#                 cv2.imwrite(os.path.join(f'../bodym-dataset/Export/images_front/Avatar_{counter:06d}.png'), silhouette)
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/Avatar_{counter:06d}.npy'), measurements)
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/height/Avatar_{counter:06d}.npy'), hwg_data[subject_id][0])
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/weight/Avatar_{counter:06d}.npy'), hwg_data[subject_id][1])
#     
#                 silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask_left/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#                 silhouette = cv2.resize(silhouette,(240, 320))
#                 cv2.imwrite(os.path.join(f'../bodym-dataset/Export/images_side/Avatar_{counter:06d}.png'), silhouette)
#                 writer.writerow([subject_id, image, measurements, hwg_data[subject_id], counter])
#                 
#                 counter += 1
#     
#     
#         
#         
# 
# 
# 
